# モデルデプロイ

とりあえずデプロイ用のモデルを作る

## Data Load

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mlp
import seaborn as sns
import numpy as np

pd.set_option('display.max_columns', 200)
plt.style.use('ggplot')

電気通信事業者の解約データを読み込む  
(https://www.kaggle.com/blastchar/telco-customer-churn)

In [2]:
input_path = '../data'
df = pd.read_csv(os.path.join(input_path, 'WA_Fn-UseC_-Telco-Customer-Churn.csv'))

# TotalCharges列に空文字が存在して文字列型になっているので欠損値に置換して少数型にしておく
col = 'TotalCharges'
df[col] = df[col].replace({' ': np.nan}).astype(float)

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Data Partition

In [3]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, train_size=0.8, random_state=2021, shuffle=True)
print('original_size:', df.shape)
print('train_size:', train_df.shape)
print('test_size:', test_df.shape)

original_size: (7043, 21)
train_size: (5634, 21)
test_size: (1409, 21)


## Data Preparation

In [4]:
from sklearn.preprocessing import StandardScaler
from category_encoders import OrdinalEncoder

In [5]:
# ターゲットを変換
train_df['Churn'] = train_df['Churn'].map({'Yes':1, 'No':0})
test_df['Churn'] = test_df['Churn'].map({'Yes':1, 'No':0})

# 数値変数の標準化
# 今回はツリー系アルゴリズムを使用する本来は不要だが、勉強のために実施しておく
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
scaler = StandardScaler()
train_df[num_cols] = scaler.fit_transform(train_df[num_cols])
test_df[num_cols] = scaler.transform(test_df[num_cols])

# customerID以外のカテゴリ変数をOrdinalエンコーディング
cat_cols = []
for col in train_df.columns:
    if train_df[col].dtype == 'object':
        cat_cols.append(col)
cat_cols.remove('customerID')
encoder = OrdinalEncoder()
train_df[cat_cols] = encoder.fit_transform(train_df[cat_cols])
test_df[cat_cols] = encoder.transform(test_df[cat_cols])

# 欠損値をトレーニングデータの中央値で保管
train_df.fillna(train_df.median(), inplace=True)
test_df.fillna(train_df.median(), inplace=True)

# 前処理後のデータプレビュー
train_df.head()

/var/folders/v6/1f2j3j154zj9g03clj2_yt_00000gn/T/ipykernel_7810/1495117370.py:23: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_df.fillna(train_df.median(), inplace=True)
/var/folders/v6/1f2j3j154zj9g03clj2_yt_00000gn/T/ipykernel_7810/1495117370.py:24: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test_df.fillna(train_df.median(), inplace=True)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
6125,0871-URUWO,1,0,1,1,-0.791990,1,1,1,1,1,1,1,1,1,1,1,1,1.255598,-0.403651,1
6958,3078-ZKNTS,2,0,1,2,-0.791990,1,2,2,2,2,2,2,2,2,2,1,2,-1.485131,-0.894834,0
4062,1915-IOFGU,2,0,2,1,-1.280574,1,2,1,1,1,3,1,3,3,1,2,3,0.200833,-0.972643,1
5298,5647-FXOTP,2,1,1,1,1.121629,1,1,1,1,3,1,1,1,1,1,1,3,1.376855,1.822967,0
1214,9866-QEVEE,1,0,2,1,-0.547698,1,1,1,1,1,3,1,3,1,1,1,2,0.715758,-0.327057,1


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5634 entries, 6125 to 1140
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        5634 non-null   object 
 1   gender            5634 non-null   int64  
 2   SeniorCitizen     5634 non-null   int64  
 3   Partner           5634 non-null   int64  
 4   Dependents        5634 non-null   int64  
 5   tenure            5634 non-null   float64
 6   PhoneService      5634 non-null   int64  
 7   MultipleLines     5634 non-null   int64  
 8   InternetService   5634 non-null   int64  
 9   OnlineSecurity    5634 non-null   int64  
 10  OnlineBackup      5634 non-null   int64  
 11  DeviceProtection  5634 non-null   int64  
 12  TechSupport       5634 non-null   int64  
 13  StreamingTV       5634 non-null   int64  
 14  StreamingMovies   5634 non-null   int64  
 15  Contract          5634 non-null   int64  
 16  PaperlessBilling  5634 non-null   int64

## Modeling

LightGBMでモデルを作成する  
バリデーションスキームはLeave One Outで、ランダムに8:2に分割する

In [6]:
import lightgbm as lgb

In [8]:
# ターゲットと特徴量に分離
train_x = train_df.drop(['customerID','Churn'], axis=1)
train_y = train_df['Churn']
test_x = test_df.drop(['customerID','Churn'], axis=1)
test_y = test_df['Churn']

# EarlyStopping用にさらにデータを分割する
train_x_, val_x, train_y_, val_y = train_test_split(train_x, train_y, train_size=0.8, random_state=888, shuffle=True)

# 学習用のデータ
dtrain = lgb.Dataset(data=train_x_, label=train_y_)
# EarlyStoppingのためのデータ
dval = lgb.Dataset(data=val_x, label=val_y, reference=dtrain)

# ハイパーパラメータ設定
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'learning_rate': 0.02,
    'num_leaves': 15,
    'bagging_freq': 5,
    'bagging_fraction': 0.8,
    'feature_fraction': 0.8,
    'random_state': 777,
    'min_data_in_leaf': 1,
    'verbose': -1
}

# モデルの学習実行
model = lgb.train(
    params, dtrain, num_boost_round=9999, valid_sets=dval, early_stopping_rounds=200
)

[1]	valid_0's binary_logloss: 0.577526
Training until validation scores don't improve for 200 rounds
[2]	valid_0's binary_logloss: 0.572274
[3]	valid_0's binary_logloss: 0.567982
[4]	valid_0's binary_logloss: 0.563359
[5]	valid_0's binary_logloss: 0.558653
[6]	valid_0's binary_logloss: 0.554153
[7]	valid_0's binary_logloss: 0.550236
[8]	valid_0's binary_logloss: 0.546518
[9]	valid_0's binary_logloss: 0.542333
[10]	valid_0's binary_logloss: 0.538431
[11]	valid_0's binary_logloss: 0.534706
[12]	valid_0's binary_logloss: 0.531111
[13]	valid_0's binary_logloss: 0.528088
[14]	valid_0's binary_logloss: 0.525296
[15]	valid_0's binary_logloss: 0.522425
[16]	valid_0's binary_logloss: 0.519348
[17]	valid_0's binary_logloss: 0.516887
[18]	valid_0's binary_logloss: 0.514061
[19]	valid_0's binary_logloss: 0.511301
[20]	valid_0's binary_logloss: 0.508807
[21]	valid_0's binary_logloss: 0.506329
[22]	valid_0's binary_logloss: 0.50411
[23]	valid_0's binary_logloss: 0.501851
[24]	valid_0's binary_loglos

In [9]:
# EarlyStoppingでトレーニングを打ちとめた時点でのAUCを計算
from sklearn.metrics import roc_auc_score

val_pred = model.predict(val_x)
print('validation_score(AUC):', roc_auc_score(y_true=val_y, y_score=val_pred))

validation_score(AUC): 0.8311496770480271


## LightGBMオリジナル形式でテスト用データの推論実行

In [10]:
# テスト用データでのAUCを計算
test_pred = model.predict(test_x)

print('test_score(AUC):', roc_auc_score(y_true=test_y, y_score=test_pred))

test_score(AUC): 0.848412674294339


In [11]:
# 結果を格納しておく
results = {}

result = {
    'model': 'LightGBM',
    'validation_score(AUC)': roc_auc_score(y_true=val_y, y_score=val_pred),
    'test_score(AUC)': roc_auc_score(y_true=test_y, y_score=test_pred)
}
results['Manually'] = result

results

{'Manually': {'model': 'LightGBM',
  'validation_score(AUC)': 0.8311496770480271,
  'test_score(AUC)': 0.848412674294339}}

In [20]:
# テスト用のデータを出力しておく
test_x.to_csv(os.path.join(input_path, 'test_x.csv'), encoding='utf-8', index=False)
test_y.to_csv(os.path.join(input_path, 'test_y.csv'), encoding='utf-8', index=False)

## デプロイ用にモデルをエクスポート

作成したLightGBMのモデルをONNX形式でエクスポート  
推論サーバで動かせるようにする

In [12]:
# LightGBMのモデルをONNX形式でエクスポートするためのライブラリを読み込む
import onnxmltools
from onnxmltools.convert.common.data_types import FloatTensorType

In [13]:
# ONNX形式でエクスポート
initial_types = [['inputs', FloatTensorType([None, len(train_x.columns)])]]
onnx_model = onnxmltools.convert_lightgbm(model, initial_types=initial_types, target_opset=9)

# ノートブック実行ディレクトリの直下に出力したlgb.onnxを、後ほど推論サーバに配置します
# 推論サーバ構築作業を行うディレクトリへ直接出力することもできますが、今回はデプロイ作業感を味わいたいので、あえて行っていません
onnxmltools.utils.save_model(onnx_model, 'lgb.onnx')

## エクスポートしたモデルの動作確認

In [15]:
# ONNX形式のモデルを動かすためのライブラリを読み込む
import onnxruntime as rt

# ONNX形式で推論
session = rt.InferenceSession("lgb.onnx")

# 入力値の設定
input_name = session.get_inputs()[0].name
np_data = np.array(test_x.values).astype(np.float32)

# テスト用データでのAUCを計算
pred_list = []
for i in range(len(test_x)):
    onnx_pred = session.run(None, {input_name: np_data[i,:].reshape(1,-1)})
    pred_list.append(onnx_pred[1][0][1])

test_pred_onnx = np.array(pred_list)

print('test_score(AUC):', roc_auc_score(y_true=test_y, y_score=test_pred_onnx))

test_score(AUC): 0.848412674294339


## オリジナル形式とONNX形式で推論結果を比較

In [16]:
test_pred

array([0.31988446, 0.05443292, 0.22777781, ..., 0.03101939, 0.62259999,
       0.45430846])

In [17]:
test_pred_onnx

array([0.31988442, 0.05443287, 0.22777778, ..., 0.03101945, 0.62260008,
       0.45430845])

ONNXランタイムは倍精度浮動小数点ではなく単精度浮動小数点を使用するため、わずかな誤差が生じることがある

## オリジナル形式とONNX形式の推論速度比較

In [104]:
%%timeit -r 7 -n 1000

# LightGBMオリジナル形式の推論速度計測
test_pred = model.predict(test_x.values[0,:].reshape(1,-1))

190 µs ± 21.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [105]:
%%timeit -r 7 -n 1000

# ONNX形式の推論速度計測
onnx_pred = session.run(None, {input_name: np_data[0,:].reshape(1,-1)})

36.8 µs ± 5.23 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## 推論サーバのコンテナ起動

ここまで来たらノートブック上での作業は一時停止  
資料のハンズオンの進め方に戻って推論サーバのコンテナを起動する

## web APIとしてデプロイしたモデルで推論実行

推論サーバのコンテナが起動したら、再びこのノートブックに戻ってきてリクエストを投げる

In [19]:
%%bash

curl web_single_pattern:8000/health

{"health":"ok"}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    15  100    15    0     0    937      0 --:--:-- --:--:-- --:--:--   937


In [20]:
%%bash

curl web_single_pattern:8000/metadata

{"data_type":"float32","data_structure":"(1,19)","data_sample":[[1.0,0.0,1.0,1.0,-0.79199,1.0,1.0,3.0,3.0,1.0,3.0,1.0,3.0,1.0,1.0,1.0,3.0,0.04635515,-0.60534847]],"prediction_type":"float32","prediction_structure":"(1,2)","prediction_sample":[0.6801156401634216,0.31988435983657837]}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   283  100   283    0     0  15722      0 --:--:-- --:--:-- --:--:-- 15722


In [21]:
%%bash

curl web_single_pattern:8000/label

{"0":"not churn","1":"churn"}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    29  100    29    0     0   2636      0 --:--:-- --:--:-- --:--:--  2636


In [22]:
%%bash

curl web_single_pattern:8000/predict/test

{"prediction":[0.6801156401634216,0.31988435983657837]}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    55  100    55    0     0   5000      0 --:--:-- --:--:-- --:--:--  5000


推論結果として戻ってくるリストの1つ目の要素はラベル0の確率、2つ目の要素はラベル1の確率

In [18]:
%%bash

curl \
    -X POST \
    -H "Content-Type: application/json" \
    -d '{"data": [[2,0,2,1,-1.199142974743425,2,3,3,3,1,3,3,3,3,1,2,4,-1.0034271946515967,-0.9422390569253819]]}' \
    web_single_pattern:8000/predict

{"prediction":[0.7722222208976746,0.22777777910232544]}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   159  100    55  100   104   3666   6933 --:--:-- --:--:-- --:--:-- 10600


In [109]:
# テスト用データ全件について推論実行
import requests

headers = {"Content-Type": "application/json"}

pred_list = []
for i in range(len(test_x)):
    request_data = np_data[i,:].reshape(1,-1)
    data = {"data": request_data.tolist()}
    try:
        r = requests.post('http://web_single_pattern:8000/predict', json=data, headers=headers)
        pred_list.append(list(r.json().values())[0][1])
    except:
        pred_list.append(0.5) # 通信の問題か入力の問題か、結果が返ってこないことがあったので補完

test_pred_deploy = np.array(pred_list)

# テスト用データのAUCを計算
print('test_score(AUC):', roc_auc_score(y_true=test_y, y_score=test_pred_deploy))

test_score(AUC): 0.848412674294339


## オリジナル形式とONNX形式（ノートブック）とONNX形式（web API）で推論結果を比較

In [16]:
test_pred

array([0.31988446, 0.05443292, 0.22777781, ..., 0.03101939, 0.62259999,
       0.45430846])

In [17]:
test_pred_onnx

array([0.31988442, 0.05443287, 0.22777778, ..., 0.03101945, 0.62260008,
       0.45430845])

In [108]:
test_pred_deploy

array([0.31988436, 0.05443287, 0.22777778, ..., 0.03101945, 0.62260002,
       0.45430845])

ONNXランタイムは倍精度浮動小数点ではなく単精度浮動小数点を使用するため、わずかな誤差が生じることがある